In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,399 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,678 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://ppa.launchpadcontent.net/c2d4u.team/c2d4

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=cb0e3f31182ed6c75ce6ce2fcc1511375848308306d68838955b9fc330d57f81
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, ignoreLeadingWhiteSpace=True)
home_sales_df.show(5)



+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [5]:
## Take a look at the schema
home_sales_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [6]:
## Fix home_sales_df struct fields
# Import struct fields
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

# Create list of struct fields
schema = [StructField("id", StringType(), True), StructField("date", StringType(), True), StructField("date_built", IntegerType(), True), StructField("price", IntegerType(), True), StructField("bedrooms", IntegerType(), True), StructField("bathrooms", IntegerType(), True), StructField("sqft_living", IntegerType(), True), StructField("floors", IntegerType(), True), StructField("waterfront", IntegerType(), True), StructField("view", IntegerType(), True),]
schema


[StructField('id', StringType(), True),
 StructField('date', StringType(), True),
 StructField('date_built', IntegerType(), True),
 StructField('price', IntegerType(), True),
 StructField('bedrooms', IntegerType(), True),
 StructField('bathrooms', IntegerType(), True),
 StructField('sqft_living', IntegerType(), True),
 StructField('floors', IntegerType(), True),
 StructField('waterfront', IntegerType(), True),
 StructField('view', IntegerType(), True)]

In [7]:
## Pass in our fields
final = StructType(fields=schema)
final

StructType([StructField('id', StringType(), True), StructField('date', StringType(), True), StructField('date_built', IntegerType(), True), StructField('price', IntegerType(), True), StructField('bedrooms', IntegerType(), True), StructField('bathrooms', IntegerType(), True), StructField('sqft_living', IntegerType(), True), StructField('floors', IntegerType(), True), StructField('waterfront', IntegerType(), True), StructField('view', IntegerType(), True)])

In [8]:
## Read our data with our new schema
dataframe = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), schema=final, sep=",", header=True)
dataframe.show()

+--------------------+----------+----------+------+--------+---------+-----------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167| 11733|         2|   1|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235| 14384|         1|   0|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127| 10575|         2|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631| 11149|         2|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249| 13878|         2|   0|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       1965| 14375|         2|   0|
|131492a1-72e2-4a8...|2020-02-08|      2017|41

In [9]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales')


In [10]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
## Imports for date/time
from pyspark.sql import SQLContext
from pyspark.sql.functions import round, avg


spark.sql("""
SELECT round(avg(price), 2) AS avg_price, EXTRACT(YEAR FROM date) AS year
FROM home_sales
WHERE bedrooms = 4
GROUP BY year
ORDER BY year
""").show()



+---------+----+
|avg_price|year|
+---------+----+
| 300263.7|2019|
|298353.78|2020|
|301819.44|2021|
|296363.88|2022|
+---------+----+



In [11]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
SELECT round(avg(price), 2) AS avg_price, date_built
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
""").show()


+---------+----------+
|avg_price|date_built|
+---------+----------+
|292859.62|      2010|
|291117.47|      2011|
|293683.19|      2012|
|295962.27|      2013|
|290852.27|      2014|
| 288770.3|      2015|
|290555.07|      2016|
|292676.79|      2017|
+---------+----------+



In [12]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
SELECT round(avg(price), 2) AS avg_price, date_built
FROM home_sales
WHERE bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
""").show()



+---------+----------+
|avg_price|date_built|
+---------+----------+
|285010.22|      2010|
|276553.81|      2011|
|307539.97|      2012|
|303676.79|      2013|
|298264.72|      2014|
|297609.97|      2015|
| 293965.1|      2016|
|280317.58|      2017|
+---------+----------+



In [13]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT view, round(avg(price), 2) AS avg_price
FROM home_sales
WHERE price >= 350000
GROUP BY view
""").show()

uncached_runtime = ("--- %s seconds ---" % (time.time() - start_time))
print(uncached_runtime)

+----+----------+
|view| avg_price|
+----+----------+
|   7| 403005.77|
|  51| 788128.21|
|  15|  404673.3|
|  54| 798684.82|
|  11| 399548.12|
|  29| 397771.65|
|  69| 750537.94|
|  42|  396964.5|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|   3|  398867.6|
|  30|  397862.0|
|  34| 401419.75|
|  59|  791453.0|
|   8| 398592.71|
|  28| 402124.62|
|  22| 402022.68|
|  85|1056336.74|
|  16| 399586.53|
+----+----------+
only showing top 20 rows

--- 0.7501852512359619 seconds ---


In [14]:
# Parse out the amount of seconds it took to run the previous query by turning the entire output into a string, splitting it to find just the seconds, and turn the seconds into a float.
uncached_runtime_parse = str(uncached_runtime)
split_uncached = uncached_runtime_parse.split()
seconds = split_uncached[1]
uncached_runtime_seconds = float(seconds)
uncached_runtime_seconds

0.7501852512359619

In [15]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [16]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [17]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000.

start_time = time.time()

spark.sql("""
SELECT view, round(avg(price), 2) AS avg_price
FROM home_sales
WHERE price < 350000
GROUP BY view
ORDER BY view DESC
""").show()

cached_runtime = ("--- %s seconds ---" % (time.time() - start_time))
print(cached_runtime)


+----+---------+
|view|avg_price|
+----+---------+
|   9|236299.02|
|   8|234155.36|
|   7|239960.92|
|   6|233480.09|
|  50|237517.97|
|   5|230727.41|
|  49|238078.42|
|  48|234616.34|
|  47|240530.23|
|  46|238128.72|
|  45|232313.97|
|  44|233404.96|
|  43| 237204.1|
|  42|235743.47|
|  41|235149.62|
|  40|238056.82|
|   4|234796.78|
|  39|232603.52|
|  38|239252.42|
|  37| 236525.8|
+----+---------+
only showing top 20 rows

--- 0.7251410484313965 seconds ---


In [18]:
# Parse out the amount of seconds it took to run the previous query by turning the entire output into a string, splitting it to find just the seconds, and turn the seconds into a float.
cached_runtime_parse = str(cached_runtime)
split_cached = cached_runtime_parse.split()
seconds = split_cached[1]
cached_runtime_seconds = float(seconds)
cached_runtime_seconds

0.7251410484313965

In [19]:
# Determine the runtime and compare it to uncached runtime.
print(f"The uncached runtime was {uncached_runtime_seconds} seconds and the cached runtime was {cached_runtime_seconds} seconds. This is a difference of {uncached_runtime_seconds - cached_runtime_seconds} seconds.")

The uncached runtime was 0.7501852512359619 seconds and the cached runtime was 0.7251410484313965 seconds. This is a difference of 0.02504420280456543 seconds.


In [20]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [21]:
# 11. Read the parquet formatted data.
p_home_sales_p=spark.read.parquet('home_sales_partitioned')

In [22]:
# 12. Create a temporary table for the parquet data.
p_home_sales_p.createOrReplaceTempView('home_sales_partitioned_p')

In [23]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.

start_time = time.time()

spark.sql("""
SELECT view, round(avg(price), 2) AS avg_price
FROM home_sales_partitioned_p
WHERE price < 350000
GROUP BY view
ORDER BY view DESC
""").show()

parquet_runtime = ("--- %s seconds ---" % (time.time() - start_time))
print(parquet_runtime)

+----+---------+
|view|avg_price|
+----+---------+
|   9|236299.02|
|   8|234155.36|
|   7|239960.92|
|   6|233480.09|
|  50|237517.97|
|   5|230727.41|
|  49|238078.42|
|  48|234616.34|
|  47|240530.23|
|  46|238128.72|
|  45|232313.97|
|  44|233404.96|
|  43| 237204.1|
|  42|235743.47|
|  41|235149.62|
|  40|238056.82|
|   4|234796.78|
|  39|232603.52|
|  38|239252.42|
|  37| 236525.8|
+----+---------+
only showing top 20 rows

--- 0.3827171325683594 seconds ---


In [24]:
# Parse out the amount of seconds it took to run the previous query by turning the entire output into a string, splitting it to find just the seconds, and turn the seconds into a float.
parquet_runtime_parse = str(parquet_runtime)
split_parquet = parquet_runtime_parse.split()
seconds = split_parquet[1]
parquet_runtime_seconds = float(seconds)
parquet_runtime_seconds

0.3827171325683594

In [25]:
# Determine the runtime and compare it to the cached version.
print(f"The cached runtime was {cached_runtime_seconds} seconds and the parquet runtime was {parquet_runtime_seconds} seconds. This is a difference of {cached_runtime_seconds - parquet_runtime_seconds} seconds.")

The cached runtime was 0.7251410484313965 seconds and the parquet runtime was 0.3827171325683594 seconds. This is a difference of 0.3424239158630371 seconds.


In [26]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [27]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False